In [2]:
import numpy as np
import math 

In [3]:
L, d_k, d_v = 4,8,8

q=np.random.randn(L,d_k)
k=np.random.randn(L,d_k)
v=np.random.randn(L,d_v)
print("V\n", v)

V
 [[-0.54636056 -0.15054645  0.25871746  0.59966612 -1.54475043 -0.67852663
  -1.53219383  1.15456754]
 [-1.79208351 -1.21408589 -1.02846623 -1.34557521  0.27998331  1.27477948
  -0.11694454 -0.14395123]
 [-0.36151134  0.83264214  1.40804037 -1.25752191 -1.81624049  0.58136874
  -1.88049262  0.92802504]
 [ 0.93952905 -1.95460459 -0.93223817 -0.62001563 -0.53428377 -0.3655168
  -1.52143447 -0.15949322]]


In [4]:
scaled=np.matmul(q,k.T) / math.sqrt(d_k)
#without mask
print(scaled)

#dk sqrt needed to reduce variance
print(q.var(), k.var(), scaled.var())

[[ 1.15507899  2.45148915 -1.51024025 -0.63965575]
 [-0.73086231 -1.64426785  0.988813    0.98746207]
 [ 0.170691    0.61012234  0.88708605  0.87430013]
 [-0.33151822 -1.36325015 -0.08440168 -0.76657739]]
1.2084942323232553 0.9952924737902545 1.2294349867684484


In [5]:
#masking
mask = np.tril(np.ones((L,L)))
mask[mask==0]=-np.inf
mask[mask==1]=0
print(mask)
#with mask
print(scaled+mask)


[[  0. -inf -inf -inf]
 [  0.   0. -inf -inf]
 [  0.   0.   0. -inf]
 [  0.   0.   0.   0.]]
[[ 1.15507899        -inf        -inf        -inf]
 [-0.73086231 -1.64426785        -inf        -inf]
 [ 0.170691    0.61012234  0.88708605        -inf]
 [-0.33151822 -1.36325015 -0.08440168 -0.76657739]]


In [6]:
def softmax(x):
    return(np.exp(x).T/np.sum(np.exp(x),axis=-1)).T

In [7]:
attention_enc=softmax(scaled)
attention_dec=softmax(scaled + mask)
print(attention_enc,"\n",attention_dec)

[[0.20441969 0.74738865 0.01422294 0.03396873]
 [0.07962386 0.03194157 0.44451734 0.44391724]
 [0.15105971 0.23441817 0.30922534 0.30529678]
 [0.3045119  0.10852475 0.38987521 0.19708814]] 
 [[1.         0.         0.         0.        ]
 [0.71369654 0.28630346 0.         0.        ]
 [0.21744496 0.33743642 0.44511862 0.        ]
 [0.3045119  0.10852475 0.38987521 0.19708814]]


In [8]:
new_v=np.matmul(attention_enc,v)
print(new_v)
print(v)

[[-1.42429686 -0.99272149 -0.72741752 -0.92203087 -0.15050227  0.80990414
  -0.47904093  0.13621028]
 [ 0.15562979 -0.54832569  0.19981103 -0.82945797 -1.15858405  0.08286083
  -1.63703714  0.42905453]
 [-0.32758327 -0.64660573 -0.05121693 -0.80298829 -0.89245908  0.26451627
  -1.3048517   0.37893995]
 [-0.31663299 -0.2382044   0.33239546 -0.57589708 -1.25341791  0.08634792
  -1.51227678  0.66633703]]
[[-0.54636056 -0.15054645  0.25871746  0.59966612 -1.54475043 -0.67852663
  -1.53219383  1.15456754]
 [-1.79208351 -1.21408589 -1.02846623 -1.34557521  0.27998331  1.27477948
  -0.11694454 -0.14395123]
 [-0.36151134  0.83264214  1.40804037 -1.25752191 -1.81624049  0.58136874
  -1.88049262  0.92802504]
 [ 0.93952905 -1.95460459 -0.93223817 -0.62001563 -0.53428377 -0.3655168
  -1.52143447 -0.15949322]]


In [9]:
def scaled_dot_product_attention(q,k,v, mask=None):
    d_k=q.shape[-1]
    scaled=np.matmul(q,k.T) / math.sqrt(d_k)
    if mask is not None:
        scaled=scaled+mask
    attention=softmax(scaled)
    out=np.matmul(attention,v)
    return out, attention

In [10]:
values, attention = scaled_dot_product_attention(q,k,v, mask=None)
print("Q\n", q)
print("K\n", k)
print("V\n", v)
print("new v\n", values)
print("attention\n", attention)
values2, attention2 = scaled_dot_product_attention(q,k,v, mask=mask)

print("new v\n", values2)
print("attention\n", attention2)

Q
 [[ 7.74455694e-01  1.97064575e+00 -2.53153688e+00 -5.97745676e-01
  -1.24164577e-01 -8.96424230e-01 -8.96535686e-01  1.55569502e+00]
 [ 5.71609011e-01 -2.49191484e+00  1.21663085e+00 -2.23144397e-02
  -7.59877904e-01  6.56746304e-01  3.79476681e-01 -5.34879537e-01]
 [ 3.36630491e-01  3.08335569e-01  3.11152025e-01  2.62084597e-01
  -1.61559422e+00 -1.59862759e-01 -6.39626461e-01  5.46925184e-02]
 [ 1.11802784e+00 -2.07663985e-03  5.47927395e-01  1.22459474e+00
   1.40045173e+00  5.40624961e-01  9.90553466e-01 -2.05964003e+00]]
K
 [[-0.31307806  0.37155654 -1.34929663 -0.6163584  -0.79354177 -1.01831504
   0.52439403 -0.99514255]
 [ 0.4174169   2.09491864 -0.43264611 -1.4439124  -1.09854613  0.25280404
   0.49942061  0.68256345]
 [-1.6419378  -1.19936668 -0.57121022  1.21714351 -1.76539886 -1.06801916
  -0.58612409 -1.96506796]
 [ 0.33547963  0.26765436  0.91023886 -1.2619529  -1.74859269  0.94764148
   0.54926845  0.05033437]]
V
 [[-0.54636056 -0.15054645  0.25871746  0.59966612 -1.